<a href="https://colab.research.google.com/github/rumman-adnan/Assignment-Sensors-Data/blob/main/sensors_dl_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score



In [5]:
# read data
# currently dont use this
train_data = pd.read_csv('/content/drive/MyDrive/Python work/sim_data/CSV data/training_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Python work/sim_data/CSV data/testing_data.csv')
print(train_data.shape)
print(test_data.shape)


(20000, 1002)
(10000, 1002)


In [7]:
train_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_20000_0.1_90_140_train.npy')
test_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_10000_0.1_141_178_test.npy')

print("The shape of trained data is: ",train_data.shape)
print("The shape of tested data is: ",test_data.shape)

The shape of trained data is:  (20000, 1006)
The shape of tested data is:  (10000, 1006)


In [8]:
# Extract features and target variables from the training dataset
train_sensor_data = train_data[:, :1000]
train_ID = train_data[:, 1000]
train_time = train_data[:, 1001]
train_H = train_data[:, 1002]
train_R = train_data[:, 1003]
train_S = train_data[:, 1004]
train_D = train_data[:, 1005]

# Extract features and target variables from the test dataset
test_sensor_data = test_data[:, :1000]
test_ID = test_data[:, 1000]
test_time = test_data[:, 1001]
test_H = test_data[:, 1002]
test_R = test_data[:, 1003]
test_S = test_data[:, 1004]
test_D = test_data[:, 1005]

# Display the shape of the extracted arrays to confirm extraction
train_sensor_data.shape, train_S.shape, train_D.shape, test_sensor_data.shape, test_S.shape, test_D.shape


((20000, 1000), (20000,), (20000,), (10000, 1000), (10000,), (10000,))

In [9]:
from sklearn.preprocessing import StandardScaler

# Reshape the data to fit the input requirements of a 1D CNN model
# The shape should be (num_samples, num_time_steps, num_features)
train_sensor_data_cnn = train_sensor_data.reshape(train_sensor_data.shape[0], train_sensor_data.shape[1], 1)
test_sensor_data_cnn = test_sensor_data.reshape(test_sensor_data.shape[0], test_sensor_data.shape[1], 1)

# Normalize the features
scaler = StandardScaler()
train_sensor_data_cnn = scaler.fit_transform(train_sensor_data_cnn.reshape(-1, 1)).reshape(train_sensor_data_cnn.shape)
test_sensor_data_cnn = scaler.transform(test_sensor_data_cnn.reshape(-1, 1)).reshape(test_sensor_data_cnn.shape)

# Show the shape of the reshaped and normalized data
train_sensor_data_cnn.shape, test_sensor_data_cnn.shape


((20000, 1000, 1), (10000, 1000, 1))

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the CNN model architecture
def build_cnn_model(input_shape):
    model = Sequential()

    # Convolutional layers
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2))  # Two output nodes to predict both systolic and diastolic BP

    # Compile the model
    model.compile(optimizer=Adam(), loss='mse')

    return model

# Build the CNN model
cnn_model = build_cnn_model((train_sensor_data_cnn.shape[1], 1))

# Display the model architecture
cnn_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 996, 128)          768       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 498, 128)         0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 494, 256)          164096    
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 247, 256)         0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 63232)             0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [15]:
from sklearn.metrics import mean_absolute_error

# Make predictions on the test set
predictions = cnn_model.predict(test_sensor_data_cnn)

# Separate the predictions for systolic and diastolic BP
pred_S, pred_D = predictions[:, 0], predictions[:, 1]

# Compute the MAE for systolic and diastolic BP
mae_S = mean_absolute_error(test_S, pred_S)
mae_D = mean_absolute_error(test_D, pred_D)

print(f"Mean Absolute Error for Systolic BP: {mae_S}")
print(f"Mean Absolute Error for Diastolic BP: {mae_D}")


313/313 [==============================] - 2s 5ms/step
Mean Absolute Error for Systolic BP: 159.4602992392715
Mean Absolute Error for Diastolic BP: 79.91103181238296


*Apply LSTM*